# Decorators


Decorators can be thought of as functions which modify the *functionality* of another function. They help to make your code shorter and more "Pythonic". 

To properly explain decorators we will slowly build up from functions. Make sure to restart the Python and the Notebooks for this lecture to look the same on your own computer. So lets break down the steps:

## Functions Review

In [1]:
def func():
    return 1

In [2]:
func()

1

# Scope Review
Remember from the nested statements lecture that Python uses Scope to know what a label is referring to. For example:

In [4]:
s = 'Global Variable'

def func():
    print (locals())

Remember that Python functions create a new scope, meaning the function has its own namespace to find variable names when they are mentioned within the function. We can check for local variables and global variables with the local() and globals() functions. For example:

In [5]:
print (globals())

{'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'def func():\n    return 1', 'func()', "s = 'Global Variable'\n\ndef func():\n    print locals()", "s = 'Global Variable'\n\ndef func():\n    print (locals())", 'print (globals())'], '_oh': {2: 1}, '_dh': ['/Users/hariomsingh/Documents/GitHub/Python_Lab/1.Scalar Types:int,float,None and bool'], '_sh': <module 'IPython.core.shadowns' from '/anaconda/lib/python3.6/site-packages/IPython/core/shadowns.py'>, 'In': ['', 'def func():\n    return 1', 'func()', "s = 'Global Variable'\n\ndef func():\n    print locals()", "s = 'Global Variable'\n\ndef func():\n    print (locals())", 'print (globals())'], 'Out': {2: 1}, 'get_ipython': <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x1

Here we get back a dictionary of all the global variables, many of them are predefined in Python. So let's go ahead and look at the keys:

In [6]:
print (globals().keys())

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', '_sh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'func', '_i2', '_2', '_i3', '_i4', 's', '_i5', '_i6'])


Note how **s** is there, the Global Variable we defined as a string:

In [7]:
print (globals()['s'])

Global Variable


Now lets run our function to check for any local variables in the func() (there shouldn't be any)

In [8]:
func()

{}


Great! Now lets continue with building out the logic of what a decorator is. Remember that in Python **everything is an object**. That means functions are objects which can be assigned labels and passed into other functions. Lets start with some simple examples:

###### Now we'll look at local functions, that is functions defined within the scope of other functions. We'll also look at the related concept of closures, which are key to really understanding local functions, and we'll close off the module with a look at Python's function decorators. As you'll recall, in Python the def keyword is used to define new functions. Def essentially binds the body of a function to a name in such a way that functions are simply objects like everything else in Python. It's important to remember that def is executed at runtime meaning that functions are defined at runtime. Up to now almost all of the functions we have looked at have been defined at module scope or inside classes in which case we refer to them as methods. However, Python doesn't restrict you to just defining functions in those two contexts. In fact, Python allows you to define functions inside other functions. Such functions are often referred to as local functions since they're defined local to a specific function's scope. Let's see a quick example. 

In [3]:
"""Here we define a function sort_by_last_letter, which sorts a 
list of strings by their last letter. We do this by using the 
sorted function and passing last_letter as the key function. 
Last_letter is defined inside sort_by_last_letter.
It is a local function. Let's test it out. """
def sort_by_last_letter(strings):
    def last_letter(s):
        return s[-1]
    return sorted(strings,key=last_letter)

In [21]:
# Just like module level function definitions, the definition of a local 
# function happens at runtime when the def keyword is executed. Interestingly,
# this means that each call to sort_by_last_letter results in a new definition of the function last_letter.
# That is just like any other name bound in a function body,
# last_letter is bound to separately to a new function each time it's called.
sort_by_last_letter(['hello',"myval","whytosort"])

<function sort_by_last_letter.<locals>.last_letter at 0x1060d0400>


['myval', 'hello', 'whytosort']

###### We can see this for ourselves by making a small modification to sort by last_letter to print the last_letter object. If we run this a few times, we see that indeed each execution of sort_by_last_letter results in a new last_letter instance. The main point here is that the def call in sort_by_last_letter is no different from any other name binding in the function, and a new function is created each time def is executed. Local functions are subject to the same scoping rules as other functions

In [7]:
store = []
def sort_by_last_letter(strings):
    def last_letter(s):
        return s[-1]
    store.append(last_letter)
    print(last_letter)
    return sorted(strings,key=last_letter)

In [8]:
sort_by_last_letter(['hello',"myval","whytosort"])

<function sort_by_last_letter.<locals>.last_letter at 0x105b46950>


['myval', 'hello', 'whytosort']

In [9]:
sort_by_last_letter(['hella',"myval","whytosort"])

<function sort_by_last_letter.<locals>.last_letter at 0x105b468c8>


['hella', 'myval', 'whytosort']

In [10]:
sort_by_last_letter(['hello',"myval","whytosortb"])

<function sort_by_last_letter.<locals>.last_letter at 0x105b46840>


['whytosortb', 'myval', 'hello']

###### Remember the LEGB rule for name lookup. First the local scope is checked, then the enclosing scope, next the global scope, and finally the built-in scope. This means that name lookup in local functions starts with names defined in the function itself. It proceeds to the enclosing scope, which in this case is the containing function. This enclosing scope includes both the local names of the containing function, as well as its parameters. Finally, of course, the global scope includes any module-level name bindings. You can see this in a small example.

In [23]:
""" Here we define the function inner local to outer.
Inner simply prints a global variable and a few bindings from outer. 
This example shows the essence of how the LEGB rule applies to local functions.
If you don't fully understand what's going on, you should play with this code on your own until it's clear. 
It's important to note that local functions are not members of their containing function in any way."""

g ='global'
def outer(p='param'):
    l='local'
    def inner():
        print(g,p,l)
    inner()

In [18]:
outer()

global param local


######  .  As we've mentioned, local functions are simply local name bindings in the function body. To see this, you can try to call a local function via member access syntax. The function object outer has no attribute named inner. Inner is only defined when outer is executed, and even then it's just a normal variable in the execution of the function's body. So, what are local functions useful for? As we've seen, they're useful for things like creating sorting key functions. It makes sense to define these close to the call site if they're one-off specialized functions, so local functions are a code organization in readability aid. In this way they're similar to lambdas, which as you'll recall are simple, unnamed function objects. Local functions are more general than lambdas though since they may contain multiple expressions and may contain statements such as import. Local functions are also useful for other more interesting purposes, but before we can look at those we'll need to investigate two more concepts, returning functions from functions and closures.

In [19]:
outer.inner()

AttributeError: 'function' object has no attribute 'inner'

# Functions within functions(First-Class Function)
Great! So we've seen how we can treat functions as objects, now lets see how we can define functions inside of other functions:

###### Returning Functions From Functions
###### As we've just seen, local functions are no different from any other object created inside a function's body. New instances are created for each execution of the enclosing function. They're not somehow specially bound to the enclosing function and so forth. Like other bindings in a function, local functions can also be returned from functions. Returning a local function does not look any different than returning any other object. Let's see an example. Here enclosing defines local_func and returns it. Callers of enclosing combined its returned value to a name, in this case lf, and then call it like any other function. This ability to return functions is part of the broader notion of first-class functions where functions can be passed to and returned from other functions or more generally treated like any other piece of data. This concept can be very powerful, particularly when combined with closures, which we'll explore in the next section.

In [21]:
def hello(name='Hari'):
    print ('The hello() function has been executed')
    
    def greet():
        return ('\t This is inside the greet() function')
    
    def welcome():
        return ('\t This is inside the welcome() function')
    
    print (greet())
    print (welcome())
    print ("Now we are back inside the hello() function")

In [22]:
hello()

The hello() function has been executed
	 This is inside the greet() function
	 This is inside the welcome() function
Now we are back inside the hello() function


In [23]:
welcome()

NameError: name 'welcome' is not defined

Note how due to scope, the welcome() function is not defined outside of the hello() function. Now lets learn about returning functions from within functions:
## Returning Functions

In [24]:
def hello(name='Hari'):
    
    def greet():
        return ('\t This is inside the greet() function')
    
    def welcome():
        return ("\t This is inside the welcome() function")
    
    if name == 'Hari':
        return greet
    else:
        return welcome

In [25]:
x = hello()

Now lets see what function is returned if we set x = hello(), note how the closed parenthesis means that name ahs been defined as Jose.

In [26]:
x

<function __main__.hello.<locals>.greet>

Great! Now we can see how x is pointing to the greet function inside of the hello function.

In [27]:
print (x())

	 This is inside the greet() function


Lets take a quick look at the code again. 

In the if/else clause we are returning greet and welcome, not greet() and welcome(). 

This is because when you put a pair of parentheses after it, the function gets executed; whereas if you don’t put parenthesis after it, then it can be passed around and can be assigned to other variables without executing it.

When we write x = hello(), hello() gets executed and because the name is hari by default, the function greet is returned. If we change the statement to x = hello(name = "Sam") then the welcome function will be returned. We can also do print hello()() which outputs now you are in the greet() function.

## Functions as Arguments
Now lets see how we can pass functions as arguments into other functions:

In [28]:
def hello():
    return 'Hi Hari!'

def other(func):
    print ('Other code would go here')
    print (func())

In [29]:
other(hello)

Other code would go here
Hi Hari!


Great! Note how we can pass the functions as objects and then use them within other functions. Now we can get started with writing our first decorator:

# Closures and Nested Scopes
##### So far the local functions we've looked at have all been fairly boring. They're defined within another function scope, but they don't really interact with the enclosing scope; however, we did see that local functions can reference bindings in their enclosing scope via the LEGB rule. Furthermore, we saw that local functions can be returned from their defining scope and executed in another scope. This raises an interesting question. How does a local function use bindings to objects defined in a scope that no longer exists? That is once a local function is returned from its enclosing scope, that enclosing scope is gone along with any local objects it defined. How can the local function operate without that enclosing scope? The answer is that the local function forms what is known as a closure. A closure essentially remembers the objects from the enclosing scope that the local function needs. It then keeps them alive so that when the local function is executed they can still be used. One way to think of this is that the local function closes over the objects it needs preventing them from being garbage collected. Python implements closures with a special attribute named dunder-closure. If a function closes over any objects, the net function has a dunder-closure attribute, which maintains the necessary references to those objects. We can see that in a simple example. The dunder-closure attribute of lf indicates that lf is a closure, and we can see that the closure is referring to a single object. In this case, that object is the X variable defined in the function that defined lf. So, we can see that local functions can safely use objects from an inner enclosing scope, but how is this really useful?

In [24]:
def Outer():
    x =3
    def inner(y):
        return x + y
    
i = Outer()

In [37]:
def enclosing(x):
    y = 3
    def local_func():
        print(x)
        print(y)
    return local_func

In [38]:
lf = enclosing('closed over')

In [40]:
# on calling the variables information was still remembered although we have already finish
# calling enclosing
lf()

closed over
3


In [41]:
lf.__closure__

(<cell at 0x1060cf768: str object at 0x1060dfbb0>,
 <cell at 0x1060cf798: int object at 0x10415b840>)

# Function Factories
###### A very common use for closures is in so-called function factories. These factories are functions that return other functions where the returned functions are specialized in some way based on arguments to the factory. In other words, the factory function takes some arguments. It then creates a local function, which takes its own arguments, but also uses the arguments passed to the factory. The combination of runtime function definition enclosures makes this possible. A typical example of this kind of factory creates a function, which raises numbers to a particular power. Here's how the factory looks. Raise_to takes a single argument, exp, which is an exponent. It returns a function that raises its arguments to that exponent. You can see that the local function raise_to_exp refers to exp in its implementation, and this means that Python will create a closure to refer to that object. If we call raise_to, we can verify that it creates this closure. We can also see that square does indeed behave as we expect, and we can create other functions the same way. 

In [43]:
def raise_to(exp):
    def raise_to_exp(x):
        return pow(x,exp)
    return raise_to_exp

In [45]:
square = raise_to(2)
square.__closure__

(<cell at 0x105ac7fa8: int object at 0x10415b820>,)

In [52]:
cube = raise_to(3)

In [53]:
cube(3)

27

In [54]:
cube(4)

64

In [55]:
cube.__closure__

(<cell at 0x105ac76a8: int object at 0x10415b840>,)

# The Nonlocal Keyword
###### The use of local functions raises some interesting questions regarding name lookup. We've looked in some detail at the LEGB rule, which determines how names are resolved in Python when we want the values to which those name refer. However, LEGB doesn't apply when we're making new name bindings. Consider this simple example. When we assign to message in the function local, what precisely is happening? In this case we're creating a new name binding in that function's scope for the name message to the string local. Critically we are not rebinding either of the other message variables in the code. We can see this by instrumenting the code a bit. Now we're actually calling the function's enclosing and local. Again, local is creating an entirely new name binding, which only applies in the context of that function. If we run this code, we'll see that neither the global nor enclosing bindings for message are affected by calling local. In earlier courses we discussed Python's global keyword. Global can be used to introduce a binding from the global scope into another scope. So, in our example if we wanted the function local to modify the global binding for message rather than creating a new one, we could use the global keyword to introduce the global message binding into local. Let's do that and see the effects. 

In [85]:
message = 'global'

def enclosing():
    message = 'enclosing'
    
    def local():
        message = 'local'
        
    print('enclosing message',message)
    local()
    print('enclosing message',message)

print('global message',message)
enclosing()
print('global message',message)


global message global
enclosing message enclosing
enclosing message enclosing
global message global


In [87]:
message = 'global'

def enclosing():
    message = 'enclosing'
    
    def local():
        global message
        message = 'local'
    print('enclosing message',message)
    local()
    print('enclosing message',message)

print('global message',message)
enclosing()
print('global message',message)

global message global
enclosing message enclosing
enclosing message enclosing
global message local


In [88]:
message = 'global'

def enclosing():
    message = 'enclosing'
    
    def local():
        nonlocal message
        message = 'local'
    print('enclosing message',message)
    local()
    print('enclosing message',message)

print('global message',message)
enclosing()
print('global message',message)

global message global
enclosing message enclosing
enclosing message local
global message global


###### First let's use the global keyword to introduce the module level binding of message into the function local. If we run this, we can see that the module level binding of message is indeed changed when local is called. Again, the global keyword should be familiar to you already. If it's not, you can always review Module 4 of the Python Fundamentals course. If global allows you to insert module-level name bindings into a function in Python, how can you do the same for name bindings in enclosing scopes? Or in terms of our example, how can we make the function local modify the binding for message defined in the function enclosing? The answer to that is that Python also provides the keyword nonlocal. Nonlocal inserts a name binding from an enclosing namespace into the local namespace. More precisely, nonlocal searches the enclosing namespace from innermost to outermost for the name you give it. As soon as it finds a match, that binding is introduced into the scope where nonlocal was invoked. Let's modify our example again to show how the function local can be made to modify the binding of message created in the function enclosing by using nonlocal. Now when we run this code we see that local is indeed changing the binding in enclosing. It's important to remember that it's an error to use nonlocal when there's no matching enclosing binding. If you do this, Python will raise a SyntaxError. You can see this if you add a call to nonlocal in our function local, which refers to a nonexistent name. When you try to execute this code, Python will complain that no such name does not exist. Like global, nonlocal is not something you're likely to need to use a lot, but it's important to understand how to use it for those times when it's really necessary or for when you see it used in other people's code. To really drive it home, let's create a more practical example that uses nonlocal. In this example, the make_timer function returns a new function. Each time you call this new function it returns the elapsed time since the last time you called it. Here's how it looks. (Typing) And here's how you can use it. (Typing) As you can see, the first time you invoke T it returns nothing. After that it returns the amount of time since the last invocation. How does this work? Every time you call make_timer it creates a new local variable named last_called. It then defines a local function called elapsed, which uses the nonlocal keyword to insert make_timer's binding of last_called into its local scope. Elapsed then uses the last_called binding to keep track of the last time it was called. In other words, elapsed uses nonlocal to refer to a name binding, which will exist across multiple calls to elapsed. In this way elapsed is using nonlocal to create a form of persistent storage. It's worth noting that each call to make_timer creates a new independent binding of last_called, as well as a new definition of elapsed. This means that each call to make_timer creates a new independent timer object, which you can verify by creating multiple timers. As you can see, calls to T1 have no affect on T2, and they are both keeping independent times.



In [93]:
import time 
def make_timer():
    last_called = None
    
    def elapsed():
        nonlocal last_called
        now = time.time()
        if last_called is None:
            last_called = now
            return None
        result = now - last_called
        last_called = now
        return result
    return elapsed

In [94]:
t = make_timer()
t()
t()

2.193450927734375e-05

# Function Decorators
##### Now that we've looked at the concepts of local functions and closures, we have what we need to finally look at an interesting and useful Python feature called the decorators. At a high level decorators are a way to modify or enhance existing functions in a nonintrusive and maintainable way. In Python a decorator is a callable object that takes in a callable and returns a callable. If that sounds a bit abstract, it might be simpler for now to think of decorators as functions that take a function as an argument and return another function, but the concept is a bit more general than that as we'll see. Coupled with this definition is a special syntax that lets you decorate functions with decorators. The syntax looks like this. This example applies the decorator, in this case named @my_decorator, to the function named in this case my_function(). The at symbol is a special syntax for applying decorators to functions. So, what does this actually do? When Python sees decorator application like this, it first compiles the base function, which in this case is my_function. As always, this produces a new function object. Python then passes this function object to the function my_decorator. Remember that decorators by definition take callable objects as their only argument, and they're required to return a callable object as well. After calling the decorator with the original function object, Python takes the return value from the decorator and binds it to the name of the original function. The end result is that the name my_function is bound to the result of calling my_decorator with the function created by the def my_function line. In other words, decorators allow you to replace, enhance, or modify existing functions without changing those functions. Callers of the original function don't have to change their code because the decorator mechanism ensures that the same name is used for both the decorated and undecorated function.

## Creating a Decorator
In the previous example we actually manually created a Decorator. Here we will modify it to make its use case clear:

In [30]:
def new_decorator(func):

    def wrap_func():
        print ("Code would be here, before executing the func")

        func()

        print ("Code here will execute after the func()")

    return wrap_func

def func_needs_decorator():
    print ("This function is in need of a Decorator")

In [31]:
func_needs_decorator()

This function is in need of a Decorator


In [32]:
# Reassign func_needs_decorator
func_needs_decorator = new_decorator(func_needs_decorator)

In [33]:
func_needs_decorator()

Code would be here, before executing the func
This function is in need of a Decorator
Code here will execute after the func()


So what just happened here? A decorator simple wrapped the function and modified its behavior. Now lets understand how we can rewrite this code using the @ symbol, which is what Python uses for Decorators:

In [34]:
@new_decorator
def func_needs_decorator():
    print ("This function is in need of a Decorator")

In [35]:
func_needs_decorator()

Code would be here, before executing the func
This function is in need of a Decorator
Code here will execute after the func()


**Great! You've now built a Decorator manually and then saw how we can use the @ symbol in Python to automate this and clean our code. You'll run into Decorators a lot if you begin using Python for Web Development, such as Flask or Django!**